In [6]:
#import torch
#import torch.nn as nn
import numpy as np

In [4]:
def read_bci_data():
    S4b_train = np.load('S4b_train.npz')
    X11b_train = np.load('X11b_train.npz')
    S4b_test = np.load('S4b_test.npz')
    X11b_test = np.load('X11b_test.npz')

    train_data = np.concatenate((S4b_train['signal'], X11b_train['signal']), axis=0)
    train_label = np.concatenate((S4b_train['label'], X11b_train['label']), axis=0)
    test_data = np.concatenate((S4b_test['signal'], X11b_test['signal']), axis=0)
    test_label = np.concatenate((S4b_test['label'], X11b_test['label']), axis=0)


    train_label = train_label - 1
    test_label = test_label -1
    train_data = np.transpose(np.expand_dims(train_data, axis=1), (0, 1, 3, 2))
    test_data = np.transpose(np.expand_dims(test_data, axis=1), (0, 1, 3, 2))
   

    mask = np.where(np.isnan(train_data))
    train_data[mask] = np.nanmean(train_data)

    mask = np.where(np.isnan(test_data))
    test_data[mask] = np.nanmean(test_data)

   

    return train_data, train_label, test_data, test_label

In [12]:
train_data, train_label, test_data, test_label = read_bci_data()
print(train_data[0][0][0].size)
print(train_label.size)

750
1080


In [ ]:
class EEGNet(nn.Module):
    def __init__(self,activation):
        super(EEGNet,self).__init__()
        self.firstconv=nn.Sequential(
            nn.Conv2d(1,16,kernel_size=(1,51),stride=(1,1),padding=(0,25),bias=False),
            nn.BatchNorm2d(16,eps=1e-5,momentum=0.1,affine=True,track_running_stats=True)
        )
        self.depthwiseConv=nn.Sequential(
            nn.Conv2d(16,32,kernel_size=(2,1),stride=(1,1),groups=16,bias=False),
            nn.BatchNorm2d(32,eps=1e-5,momentum=0.1,affine=True,track_running_stats=True),
            activation,
            nn.AvgPool2d(kernel_size=(1,4),stride=(1,4),padding=0),
            nn.Dropout(p=0.25)
        )
        self.seperableConv=nn.Sequential(
            nn.Conv2d(32,32,kernel_size=(1,15),stride=(1,1),padding=(0,7),bias=False),
            nn.BatchNorm2d(32,eps=1e-5,momentum=0.1,affine=True,track_running_stats=True),
            activation,
            nn.AvgPool2d(kernel_size=(1,8),stride=(1,8),padding=0),
            nn.Dropout(p=0.25)
        )
        self.classify=nn.Linear(736,2)
    def forward(self,inputs):
        output = self.firstconv(inputs)
        output = elf.depthwiseConv(output)
        output = self.seperableConv(output)
        return self.classify(output)
def calculate_accuracy(pred_y, labels_y):
    pred_y = pred_y.reshape(-1,1)
    pred_y = np.round(pred_y) 
    correct = np.sum(pred_y == labels_y)
    total = labels_y.shape[0]  
    accuracy = (correct / total) * 100  
    return accuracy

In [3]:
Batch_size= 64        
lr = 1e-2        
Epochs = 150
activations = ["ReLU", "LeakyRLU", "ELU"]
loss = torch.nn.CrossEntropyLoss()
train_acc = []
for act in activations:
    model=EEGNet(act)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    train = []
    for epoch in Epochs:
        for index, data in enumerate(train_data):
            predict = model(data)
            loss = loss(train_label, predict)
            accuracy = 
            if epoch % 100 == 0:
                print(f"Epoch: {epoch}, Loss: {np.mean(loss)}, Accuracy: {accuracy:.2f}%")

ReLU
LeakyRLU
ELU


NameError: name 'EEGNet' is not defined